In [1]:
%pip install tensorflow

import tensorflow as tf
import pandas as pd
from tensorflow import keras

import numpy as np
from collections.abc import Iterable
import unittest

You should consider upgrading via the 'c:\users\chris\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


# Introduction

The majority of the content in this section is built to support the dynamic architecture. This is notable from a static architecture due to possessing an additional term, mean length, which is passed through each layer in addition to any primary tensors. It is the case that dynamic tensors will modify mean length, and furthermore that loss can be applied with respect to mean length

## Architecture

This project is built, suprisingly enough, in tensorflow. Each logical layer is expected to pass forward two parameters. These are the stream and mean length. Any modifications made to the mean length must be applied as such, and in a gradient-descent friendly format. This is, suprisingly enough, also possible. Finally, layers will exist which can modify the length of embeddings traversing the encoding. Most shockingly, this is, also, possible

Notably, it is now also a requirment for higher dimensional tensors to work 


## Base

The base layer for the model. Contains a new space, "forward", to perform calculations in. Handles mean passthrough implicitly.


# Basics

Some of the basic tools I will need are defined here.

* DynTensor - A utility to handle dynamic resized tensors and track needed info.
* Base - The base layer
* Local - A view of a tensor dimension in local notation.
* View - A nice view function for modifying only the last few dimensions of a tenosr. 
* Linear - A tweaked linear layer. Capable of preserving heads
* PosEncodings - Basic positional encodings.
* Transformer - A tweaked transformer. Capable of generating, working with, and collapsing predefined heads when requested
* Feedforward - A tweaked feedforward layer. Capable of preserving heads if requested.
* SortPartition - Handle vectorized batched masks producing entries of inequivalent length. Pads, sorts, keeps longest neded length. Rest is zeros.
* LengthEncode - Takes a string of entries, and their mean lengths. Forms from each of them 


## Tensor Dynamics

Tensor Dynamics is a small microlibrary consisting of a set of tools utilized to greatly ease the management of tensors when performing both standard and dynamic operations. It is designed to greatly ease the workload of a tensor programmer by allowing named dimensions and dynamic corrolation. It has the following design philosophy:

* Relevant Dimension Policy: One should only have to worry about the relevant dimensions when performing an operatin
* Implicit Tracking: Anything which is not being explictly modified should stay the same.
* Easy Dynamics: It should be easy to form dynamic masks and dynamic thresholds

### Architecture

Tensor Dynamics consists of three primary classes. These are the DynSpec, the DynTensor, and the DynRestore. These work together to create tensor dynamics. Tensor dynamics consists of allowing the masking of particular dimensions, with respect to another dimension, in order to virtually shorten the length of a tensor. In particular, let

**DynSpec**

The DynSpec defines the dimensions of a tensor, along with any dynamic linkage which occurs. Configuration errors are not checked in this class - none


In [117]:
class DynSpec():
    """

    A DynSpec. An editable container where spec information can be kept.
    
    Does not by itself error check. Does, however, contain methods
    "insert", "add", "replace", "remove", and "front" allowing
    the easy editing of what is inside of it. To construct a spec
    this way, make sure to start it with DynSpec() - no parameters
    
    .spec contains the actual table. 
    
    """
    
    _column = ("labels", "pointers", "lengths", "param")
    def make_frame(self, label, pointer, length, param):
        """
        Makes a dataframe which can be appended or inserted as needed.
                
        """
        
        label_data = self._column
        raw_data = [[label], [pointer], [length], [param]]
        build_data = dict(zip(label_data, raw_data))
        return pd.DataFrame(build_data)
    
    @classmethod
    def create(cls, labels, pointers=None, lengths=None, params=None):
        """
        The create method. Something of an alternative to the __init__ method, create will
        take a series of lists meant to represent a spec and go create a spec out of the list.
        
        It will then return the DynSpec.
        
        For each dimension:
            label must be a string
            pointer and length must be None or (string, 1D_tensor)
            length, if 1d, must equal tensor length.            
        Across Dimensions:
            pointer must point towards another existing label.  
        
        """
        
        #Handle implicit nones
        
        if pointers is None:
            pointer = [None]*len(labels)
        if lengths is None:
            lengths = [None]*len(labels)
        if params is None:
            params = [None]*len(labels)
        
        #zip entries together, make construction dictionaries.
        
        raw_data = [labels, pointers, lengths, params]
        raw_labels = cls._column
        data = dict(zip(raw_labels, raw_data))
        
        #Create pandas frame, make DynSpec
        
        spec = pd.DataFrame(data)
        return DynSpec(spec)
        
        
        
        
    def __init__(self, spec=None):
        """
        Startup. Store a spec if available, otherwise start a generic table.
        """
        if spec is None:
            self.spec = pd.DataFrame(columns= self._column)
        else:
            self.spec = spec
    
    def front(self, label, pointer=None, length=None, param=None):
        
        #Create frame.
        frame = self.make_frame(label, pointer, length, param)
        
        #Store it at the front of the spec
        spec = pd.concat([frame, self.spec], axis=0, ignore_index=True)
        self.spec = spec
    
    def insert(self, index, label, pointer=None, length=None, param=None):
        """
        
        Insert a dimension somewhere inside the spec.
        
        """
        
        
        #Create frame
        frame = self.make_frame(label, pointer, length, param)
        
        #slice apart spec
        
        prior = self.spec.iloc[:index, :]
        post = self.spec.iloc[index:, :]
        
        #create new spec
        spec = pd.concat([prior, frame, post], ignore_index=True)
        self.spec = spec        
        
    def append(self, label, pointer=None, length=None, param=None):
        """
        Append a new dimension to the end of the spec.
        
        """
        
        #create frame
        frame = self.make_frame(label, pointer, length, param)
    
        #create spec
        spec = pd.concat([self.spec, frame], axis=0, ignore_index =True)
        
        #create return
        
        return DynSpec(spec)
    
    
    

In [41]:

class TestDynSpec(unittest.TestCase):
    def setUp(self):
        self.spec = DynSpec()
    def testFront(self):
        self.spec.front("test")
        self.spec.front("test2", "test", tf.constant([3]))
        self.spec.front("test3", param = "blink")
    def testInsert(self):
        self.spec.insert(0, "test4")
        self.spec.insert(0, "test5", param="tweak")
        self.spec.insert(1, "test6")
    def testAppend(self):
        self.spec.append("test")
        self.spec.append("test2")
        self.spec.append("test3")
    def testCreate(self):
        labels = ["a", "b", "c"]
        param = [None, "test", None]
        
        item1 = self.spec.create(labels)
        item2 = self.spec.create(labels, params = param)
                
unittest.main(argv=[''], verbosity=2, exit=False)    


testAppend (__main__.TestDynSpec) ... ok
testCreate (__main__.TestDynSpec) ... ok
testFront (__main__.TestDynSpec) ... ok
testInsert (__main__.TestDynSpec) ... ok
testIsSane (__main__.TestDynTensor) ... ERROR

ERROR: testIsSane (__main__.TestDynTensor)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\chris\AppData\Local\Temp/ipykernel_21104/393677500.py", line 52, in testIsSane
    NotTensor()
  File "C:\Users\chris\AppData\Local\Temp/ipykernel_21104/393677500.py", line 25, in NotTensor
    self.assertTrue(self.is_code(DynTensor.is_sane(None, None), 0))
AttributeError: type object 'DynTensor' has no attribute 'is_sane'

----------------------------------------------------------------------
Ran 5 tests in 0.288s

FAILED (errors=1)


In [139]:
class SpecError(Exception):
    def __init__(self, msg, code, index):
        self.code = code
        super().__init__(msg)


class DynTensor():
    """
    
    The DynTensor consists of the management engine for dynamics. A DynTensor is started
    with a spec, and a tensor. The spec must be a valid DynSpec, and the tensor must 
    match the spec. Upon being started, the DynTensor will boot up it's utility functions,
    set up it's mask function, and standby for further orders. 
    
    Among other things, it can be ordered to mask, by the rules of dynamics, 
    the current tensor, 
    
    """
        
    @staticmethod
    def is_sane(tensor, spec):
        """
        
        The primary sanity checking method. This returns true if sane, 
        and the reason not sane if false
        
                
        For each dimension:
            tensor: 
                should be tensor
            spec:
                per dimension:
                    every entry in label must be a string
                    pointer and length must be None or (string, 1D_tensor)
                Across Dimensions:
                    pointer must point towards another existing label. 
            interdependent:
                spec length should equal tensor length
                length, if 1d, must equal tensor length.            

        """
        
        #Tensor independent checks
        
        if not tf.is_tensor(tensor):
            return SpecError("Tensor was not actually a tensor", 0, None)
        
        

        #Perform base length checks.
        if len(spec.spec["labels"]) != len(tensor.shape):
            return SpecError("Tensor and Spec are of different lengths", 1, (spec.spec, tensor.shape))
        #Create primary iteration frame
        
        
        primary_iteration = pd.concat([spec.spec, pd.DataFrame({"shape" : tensor.shape})], axis=1)
        #Perform dimensional self-consistency check. The majority of the 
        #handling exists here.
        pointers = []
        for index, (label, pointer, length, name, shape) in primary_iteration.iterrows():
            #Check label property
            if type(label) != str:
                return SpecError("Label at index %s was not a string" % index, 2, index)
            
            #Check pointer, length types
            if not (type(pointer) == str or pointer is None):
                return SpecError("Pointer at label %s was neither string nor None" % label, 3, index)
            if not (tf.is_tensor(length) or length is None):
                return SpecError("Length at label %s was neither tensor nor None" % label, 4, index)
            if (pointer is not None) and (pointer not in primary_iteration["labels"].array):
                return SpecError("Pointer at label %s points to label %s, which does not exist" % (label, pointer), 5, index)
        
            #Check pointer, length compatibility
            if type(pointer) == str and not tf.is_tensor(length):
                return SpecError("Pointer at label %s was string, but length was not tensor" % label, 6, index)
            if pointer is None and length is not None:
                return SpecError("Pointer at label %s was None, but length was not None" % label, 6, index)
            
            #Check length details, and compatibity
            if length is not None:
                if len(length.shape) != 1:
                    return SpecError("Length at label %s was not a 1D tensor. It was %s d" % (label, len(length.shape)), 7,  index)
                if length.shape[0] != shape:
                    return SpecError("Length at label %s did not match tensor dimension. Got %s, but tensor is %s" % (label, length.shape[0], shape), 7, index) 

        #Return true
        
        return True
    
    def __init__(self, tensor, spec, skip_error_check=False):
        """
        The initialization method for the DynTensor
        
        The DynTensor must be fed a DynSpec, which will in turn be utilized to 
        initialize the tensor. 
             
        
        """
        #Handle sanity checking. See is_sane for details, but after this point the spec and tensor are known
        #to be compatible.
        if self.is_sane(tensor, spec) is not True and skip_error_check is not True:
            raise self.is_sane(tensor, spec)
        
        #Create instance spec. 
        
        primary_spec = pd.concat([spec.spec, pd.DataFrame({"shape" : tensor.shape})], axis=1)
        self.spec = primary_spec
        #Create focused pandas arrays
        
        dyn_spec = primary_spec[primary_spec["pointers"] != None]
        target_spec = primary_spec[primary_spec["labels"].isin(dyn_spec["pointers"])]
        
        #Create needed iteration arrays. Then perform iteration
        
        
        #dyn_lengths = dyn_spec["lengths"]        
        #target_shape = target_spec["shape"]
        #mask_frame = pd.concat([dyn_lengths, target_shape], axis=1)
        
        
        #Iterate. Generate masks. Apply. 
        #for index, (length, shape) in mask_frame.iterrows():


compile
compile
tf.Tensor([b'a' b'b' b'c'], shape=(3,), dtype=string)
compile
tf.Tensor([b'd' b'e' b'f'], shape=(3,), dtype=string)
recompile?
tf.Tensor([b'a' b'b' b'c'], shape=(3,), dtype=string)
tf.Tensor([b'a' b'b' b'c'], shape=(3,), dtype=string)
tf.Tensor([b'a' b'b' b'c'], shape=(3,), dtype=string)
tf.Tensor([b'a' b'b' b'c'], shape=(3,), dtype=string)
compile
tf.Tensor([b'a' b'b' b'c'], shape=(3,), dtype=string)


In [206]:

tensor_list = [tf.constant([[1., 2., 3.]]), tf.constant([[4., 5., 6., 7.]])]

ragged = tf.ragged.stack(tensor_list)

print(dir(ragged))
print(ragged.nested_row_lengths)
print(ragged.ragged_rank)
print(ragged[0][0])
print(tf.map_fn(lambda x : x, ragged))
print(ragged.bounding_shape(axis=2))

testa = tf.constant([3.4, 2.7, 1.2, 9.1])
@tf.function
def make_length_parallel(index):
    print(index)
    spec = tf.RaggedTensorSpec([None])
    output = tf.map_fn(fn=lambda x : tf.cast(tf.zeros([tf.cast(x, tf.int32)]), dtype=tf.float32), elems=index,
                      fn_output_signature=spec)
    return output

print(make_length_parallel(testa))


['__abs__', '__abstractmethods__', '__add__', '__and__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__div__', '__doc__', '__eq__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__invert__', '__le__', '__lt__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__nonzero__', '__or__', '__pow__', '__radd__', '__rand__', '__rdiv__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rmod__', '__rmul__', '__ror__', '__rpow__', '__rsub__', '__rtruediv__', '__rxor__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__tf_tracing_type__', '__truediv__', '__weakref__', '__xor__', '_abc_impl', '_as_graph_element', '_consumers', '_convert_values_and_partition', '_eager_value', '_from_nested_row_partitions', '_from_row_partition', '_from_variant', '_is_eager', '_nested_row_partitions', '_row_partition', '_set_shape', '_shape_invariant_to_ty

In [92]:
class TestDynTensor(unittest.TestCase):

    
    #Test sanity checking function. Fairly complex.
    
    #Basic helper functions
    def is_code(self, result, code):
        self.assertTrue(isinstance(result, SpecError), "Did not recieve error when required")
        return result.code == code
    def is_not_code(self, result, code):
        if isinstance(result, SpecError):
            if result.code == code:
                return False
        return True
    #Code by code helper functions.
    
    
    def testIsSane(self):
        
        #Create testing functions.
        
        def NotTensor():
            tensor = tf.ones([10, 3, 4])
            spec = DynSpec.create(["a", "b", "c"])
            self.assertTrue(self.is_code(DynTensor.is_sane(None, None), 0))
            self.assertTrue(self.is_not_code(tensor, spec), 0)
        def NotSameLength():
            code = 1
            
            tensor = tf.ones([10, 4, 3])
            spec_pass = DynSpec.create(["a", "b", "c"])
            spec_fail = DynSpec.create(["a", "b"])
            
            test_pass = DynTensor.is_sane(tensor, spec_pass)
            test_fail = DynTensor.is_sane(tensor, spec_fail)
            
            self.assertTrue(self.is_not_code(test_pass, code), "Same length passed when it should have failed")
            self.assertTrue(self.is_code(test_fail, code), "Same length failed when it should have passed")
        def LabelsAreString():
            code = 2
            
            tensor = tf.ones([10, 4, 3])
            spec_pass = DynSpec.create(["a", "b", "c"])
            spec_fail = DynSpec.create(["a", None, "c"])
        
            test_pass = DynTensor.is_sane(tensor, spec_pass)
            test_fail = DynTensor.is_sane(tensor, spec_fail)
            
            self.assertTrue(self.is_not_code(test_pass, code), "List of string labels failed when it shoudl pass")
            self.assertTrue(self.is_code(test_fail, code), "Labels test passed when it should of failed")
        def BadPointerType():
            code = 3
            
            tensor = tf.ones([10])
            spec_pass = DynSpec.create(["a"], ["b"], [None])
            spec_pass2 = DynSpec.create(["a"], None, None)
            spec_fail = DynSpec.create(["a"], [[]])
            
            test_pass1 = DynTensor.is_sane(tensor, spec_pass)
            test_pass2 = DynTensor.is_sane(tensor, spec_pass2)
            test_fail = DynTensor.is_sane(tensor, spec_fail)
            
            self.assertTrue(self.is_not_code(test_pass1, code), "Pointer type failure")
            self.assertTrue(self.is_not_code(test_pass2, code), "Pointer type failure")
            self.assertTrue(self.is_code(test_fail, code), "Pointer type failure")
        def BadLengthType():
            code = 4
            
            tensor = tf.ones([10])
            spec_pass_1 = DynSpec.create(["a"], ["b"], [tf.constant([3])])
            spec_pass_2 = DynSpec.create(["a"], ["b"], [None])
            spec_fail = DynSpec.create(["a"], ["b"], [3])
            
            test_pass1 = DynTensor.is_sane(tensor, spec_pass_1)
            test_pass2 = DynTensor.is_sane(tensor, spec_pass_2)
            test_fail = DynTensor.is_sane(tensor, spec_fail)
            
            self.assertTrue(self.is_not_code(test_pass1, code), "LengthType issue")
            self.assertTrue(self.is_not_code(test_pass2, code), "LengthType issue")
            self.assertTrue(self.is_code(test_fail, code), "LengthType Issue")
        def BadPointing():
            code = 5
            
            tensor = tf.ones([10, 3])
            spec_pass = DynSpec.create(["a", "b"], ["b", None], [tf.constant([10]), None])
            spec_fail = DynSpec.create(["a", "b"], ["c", None], [None, None])
            
            
            test_pass = DynTensor.is_sane(tensor, spec_pass)
            test_fail = DynTensor.is_sane(tensor, spec_fail)
            
            
            self.assertTrue(self.is_not_code(test_pass, code), "Pointing exists failed")
            self.assertTrue(self.is_code(test_fail, code), "Pointing exists failed.")
        def BadCorrolated():
            code = 6
            
            tensor = tf.ones([10, 3])
            spec_pass = DynSpec.create(["a", "b"], ["b", None], [tf.constant([10]), None])
            spec_faila = DynSpec.create(["a", "b"], ["b", None], [None, None])
            spec_failb =  DynSpec.create(["a", "b"], [None, None], [tf.constant([10]), None])
            
            test_pass = DynTensor.is_sane(tensor, spec_pass)
            test_faila = DynTensor.is_sane(tensor, spec_faila)
            test_failb = DynTensor.is_sane(tensor, spec_failb)
                        
            self.assertTrue(self.is_not_code(test_pass, code), "Corrolation failure")
            self.assertTrue(self.is_code(test_faila, code), "Corrolation failure")
            self.assertTrue(self.is_code(test_failb, code), "Corrolation failure")
        def BadLength():
            code = 7
            
            tensor = tf.ones([10, 3])
            spec_pass = DynSpec.create(["a", "b"], ["b", None], [tf.ones([10]), None])
            spec_faila =  DynSpec.create(["a", "b"], ["b", None], [tf.ones([8]), None])
            spec_failb = DynSpec.create(["a", "b"], ["b", None], [tf.constant(8), None])
            
            test_pass = DynTensor.is_sane(tensor, spec_pass)
            test_faila = DynTensor.is_sane(tensor, spec_faila)
            test_failb = DynTensor.is_sane(tensor, spec_failb)
                        
            self.assertTrue(self.is_not_code(test_pass, code), "Length Shape failure")
            self.assertTrue(self.is_code(test_faila, code), "Length Shape  failure")
            self.assertTrue(self.is_code(test_failb, code), "Length Shape failure")
            
        #Execute the tests.
    
        NotTensor()
        NotSameLength()
        LabelsAreString()
        BadPointerType()
        BadLengthType()
        BadPointing()
        BadCorrolated()
        
unittest.main(argv=[''], verbosity=2, exit=False)    


testAppend (__main__.TestDynSpec) ... ok
testCreate (__main__.TestDynSpec) ... ok
testFront (__main__.TestDynSpec) ... ok
testInsert (__main__.TestDynSpec) ... ok
testIsSane (__main__.TestDynTensor) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.308s

OK


In [87]:
frame = pd.DataFrame({"a": [1, 2,3], "b" : [4,  5, 6]})
c = pd.DataFrame({"c" : [6, 7, 8]})

pd.concat([frame, c], axis=1)
[None]*0

[]

In [39]:
class DynTensor():
    """
    The DynTensor consists of the means
    and methods needed in order to manage
    a dynamic instance. This includes
    logic to manage keyed dimension
    manipulation along with dynamic 
    masking.
    
    
    
    """
    #define properties.
    @property
    def labels(self):
        return self._labels
    @property
    def tensor(self):
        return self._tensor
    @property
    def dyn(self):
        return self._dyn
    #Define startup function.
    @staticmethod
    def create(tensor, labels):
        """
        
        Creates a new DynTensor with dynamic stream from an ordinary tensorflow
        tensor. 
        
        Additional dimensions starting with "d_" are created for each original
        dimension, and a dynlist is filled with None's to match. 
        """
        
        #For each dimension, create a dynamic mirror
        final_tensor = tensor
        for dim in tensor.shape:
            final_tensor = tf.expand_dims(final_tensor, axis=0) 
        
        #For each label, make a dynamic mirror
        
        dynamic = ["d_" + item for item in labels]
        final_labels = dynamic + labels
        
        #For each dyn, make the current length
        
        dynamic_dyn = [tf.constant([dim], dtype=tf.float32) for dim in tensor.shape]
        static_dyn = [None]*len(tensor.shape)
        final_dyn = dynamic_dyn + static_dyn
        
        #Create and return the DynTensor. Do not screw this up.
        
        return DynTensor(final_tensor, final_labels, final_dyn)
        
    
    
    
    #Define primary functions
    def compress(self, final_labels):
        """
        
        This is a core logical method
        
        The compress method will reorder the dimensions of the input tensors to be in the order
        given in labels. It will do this for the tensor and the dynlist. It will then compress
        away the non-accessed dimensions into a single dimension at the front of the tensor, 
        and return four items: 
        
        tensor, labels, dyn, restore
        
        Tensor indicates the compressed tensor, and may be directly manipulated. labels are the labels
        which are actually in use. dyn contains any dynamic elements. restore, when run with 
        a tensor label pair, or tensor-label-dyn pair, will restore the compressed elements.
        """
        
        
        #Reorder in the proper sequence. 
        tensor, labels, dyn = self._reorder(final_labels)
        
        #Separate shape info, labels info, dyn info into two piles. One goes into restoration.
        #The other will be used for reshape and return.
        
        slice_length = len(final_labels)
        
        implicit_dims, explicit_dims = tensor.shape[:-slice_length], tensor.shape[-slice_length:]
        implicit_labels, explicit_labels = labels[:-slice_length], labels[-slice_length:]
        implicit_dyn, explicit_dyn  = dyn[:-slice_length], dyn[-slice_length:]
        
        #Use the implicit portions to form a restore class. Use the explicit portions to reshape the tensor
        
        shape = [-1, *explicit_dims]
        final_tensor = tf.reshape(tensor, shape)
        
        restore = DynRestore(implicit_dims, implicit_labels, implicit_dyn)
        
        #Return the compressed properties
        return final_tensor, explicit_labels, explicit_dyn, restore
        
    def mask(self, fill=0):
        """
        This has the effect of performing a filled mask of the internal quantities. 
        
        The Dyn quantity of the DynTensor is read, and should be a float. If the float is less than the current dimension,
        the portion is included. 
        
        
        """
        
        #Create index pairs. These are pairs of dimensional indices which are known to be corrolated.
        
        lookup_index = dict(zip(self.labels, range(len(self.labels))))
        dynamics = [item for item in self.labels if item.startswith("d_")]
        statics = [item[2:] for item in dynamics]
        
        
        
        
   
        #Create construction mesh. This consists of, per dimension, an indication
        #of how far above the zero index the current dimension lies.
        
        grid_make = [tf.cast(tf.range(dim), tf.float32) for dim in self.tensor.shape]
        grid = tf.meshgrid(*grid_make)
        
        
        
        #Create mask operator. Do this by taking the grid, considering each dimension, and putting together
        #a 1, 0 mask that can be multiplied.
        intermediary = []
        for grid_item, dyn_item in zip(grid, self.dyn):
            if dyn_item is not None:
                result = tf.where(grid_item <= dyn_item, 1., 0.)
                intermediary.append(result)
        intermediary = tf.stack(intermediary, axis=0)
        mask = tf.reduce_prod(intermediary, axis=0)
        
        #Apply mask effect. 
        
        tensor = self._tensor * mask
        
        #Construct new DynTensor. Return it
        
        return DynTensor(tensor, self.labels, self.dyn)
        
 
    def _reorder(self, labels):
        """
        
        A core function, this reorders the tensor, names, and dynlist to be in a 
        particular order, implicitly retaining order on the rest of the dimensions. 
        
        It then returns the result of these reorders
        
        """
        
        #Figure out what the new, reordered labels will look like.
        
        implicit_labels = []
        for item in self.labels:
            if item not in labels:
                implicit_labels.append(item)
        
        final_labels = implicit_labels + labels
        
        #Create transformation lookup tables. These will go ahead and tells us what index needs to end up
        #where, or be used directly for transformations.
        
        lookup_indices = dict(zip(self.labels, range(len(self.labels))))
        lookup_dyn = dict(zip(self.labels, self.dyn))
        
        #Create permuter. This is used by tensorflow to rearrange the dimensions.
        
        permute = [lookup_indices[item] for item in final_labels]
        
        #Perform final transforms.
        
        final_tensor = tf.transpose(self.tensor, permute)
        final_labels = final_labels
        final_dyn = [lookup_dyn[item] for item in final_labels]
        
      
        #Return results.
        return final_tensor, final_labels, final_dyn
        
        
        
    def __init__(self, tensor, dim_labels, dyn):
        
        
        #Perform sanity checking. Dyn entries must be width of dimensions or None, and 
        #tensor dims, dim_labels, and dyn must be the same length.
        
        assert len(tensor.shape) == len(dim_labels), "Label length and tensor dimensons do not match"
        assert len(dim_labels) == len(dyn), "Label length and dyn length do not match."

        for dim, item_dyn, label in zip(tensor.shape, dyn, dim_labels):
            if item_dyn is not None:
                assert tf.is_tensor(item_dyn), "Item in dyn at position %s was not None or tf Tensor" % label
                assert len(item_dyn.shape) == 1, "Item %s in dyn was not 1d" % label
                assert item_dyn.shape[0] == dim, "Dyn Tensor %s was length %s while tensor dim was %s" % (label, item_dyn.shape[0], dim)
                assert item_dyn.dtype == tf.float32, "Dyn value was not equal to float32"
        #Storage
        self._tensor = tensor
        self._labels = dim_labels
        self._dyn = dyn    
        
    

class DynRestore():
    """
    The DynRestore class is designed to restore collapsed dimensions.
    
    When provided with just a tensor with labels, it will use the final dimension,
    which is expected to be unlabeled, to attempt to re-expand the problem back into
    it's original format. It will also reconstruct the dynlist to match the incoming tensors. 
    
    When provided with a DynList as well, it will use that to assign new dynamic masks, rather than 
    an implicit fill. If 
    """
    def __init__(self, implicit_dims, implicit_labels, implicit_dyn):
        
        self._implicit_dims = implicit_dims
        self._implicit_labels = implicit_labels
        self._implicit_dyn = implicit_dyn
        
    def __call__(self, tensor, labels, dyn=None):
        """
        
    
        
        """
        
        #Construct the DynList if not given
        
        if dyn is None:
            dyn = [None]*len(labels)
            
            
        #Concatenate together various lists and quantities
        
        final_dims = [*self._implicit_dims, *tensor.shape[1:]]
        final_labels = [*self._implicit_labels, *labels]
        final_dyn = [*self._implicit_dyn, *dyn]
        
        
        #Create a shape for reshape. Execute. Then create DynTensor and return
        
        shape = final_dims
        final_tensor = tf.reshape(tensor, shape)
        
        return DynTensor(final_tensor, final_labels, final_dyn)


    

In [192]:
#Test 
class TestDynamic(unittest.TestCase):
    def setUp(self):
        """
        Create source dynamic tensor for testing. Also, see if basic setup works
        
        """
        tensor = tf.ones([10, 4, 5, 3,2])
        label = ["batch", "row", "column", "item", "channel"]
        self._dyn_tensor = DynTensor.create(tensor, label)
        self._labels = label
    def testSetup(self):
        """
        
        See if the set up values are sane
        
        """
        
        dyn_tensor = self._dyn_tensor
        label = ["d_batch", "d_row", "d_column", "d_item", "d_channel"] + self._labels
        
        self.assertEqual(dyn_tensor.tensor.shape, [1, 1, 1, 1, 1, 10, 4, 5, 3, 2]) #one additional for each dynamic
        self.assertEqual(dyn_tensor.labels, label) #Includes dynamic channels
        
        for label_item, dyn_item in zip(dyn_tensor.labels, dyn_tensor.dyn):
            if dyn_item is not None:
                self.assertTrue(tf.is_tensor(dyn_item))
                self.assertTrue(dyn_item.dtype == tf.float32)
        
    def testCompression(self):
        """
        
        Test a compression instance
        
        """
        dyn_tensor = self._dyn_tensor
        selection = ["d_batch", "batch", "row", "channel"]
        tensor, labels, dyn, restore = dyn_tensor.compress(selection)
        
        self.assertEqual(tensor.shape[0], 15, "Final shape improper")
        self.assertEqual(labels, selection, "Returned labels and selection did not match")
    def testDirectRestore(self):
        """
        Tests direct restore. Do I get back out what I put in? 
        
        """
        
        #Primary logic
        dyn_tensor = self._dyn_tensor
        selection = ["item","channel"]
        tensor, labels, dyn, restore = dyn_tensor.compress(selection)
        restore_tensor = restore(tensor, labels, dyn)
        
        #Tests
        self.assertEqual(dyn_tensor.tensor.shape, restore_tensor.tensor.shape, "Tensor shapes inequal")
        self.assertEqual(dyn_tensor.labels, restore_tensor.labels, "Labels inequal")
        self.assertEqual(dyn_tensor.dyn, restore_tensor.dyn, "Dyn inequal")
    def testResizeRestore(self):
        """
        
        Test resize restore. If I fiddle with the provided dimensions, can I still restore the others?
        
        """
        dyn_tensor = self._dyn_tensor
        selection = ["d_batch", "item", "row"]
        tensor, labels, dyn, restore = dyn_tensor.compress(selection)
        
        #Expand.
        
        expand_tensor = tf.expand_dims(tensor, axis=-1)
        expand_tensor = tf.repeat(expand_tensor, 3, axis=-1)
        labels.append("local")
        dyn.append(None)
        
        #Try 

        restore(expand_tensor, labels)
        restore(expand_tensor, labels, dyn)
    def testMask(self):
        """
        
        Test whether the mask function is working sanely
        
        """
        
        dyn_tensor = self._dyn_tensor
        
        #Static test. No masking
        masked_tensor = dyn_tensor.mask() 
        self.assertTrue(tf.reduce_all(dyn_tensor.tensor == masked_tensor.tensor))
        
        #Setup mask. Check effectiveness
        
        tensor, labels, dyn, restore = dyn_tensor.compress(["d_batch"])
        print(tf.reduce_sum(dyn_tensor.tensor))
        dyn[0] = tf.constant([5.])
        masked_tensor = restore(tensor, labels, dyn)
        print(masked_tensor.dyn)
        print(masked_tensor.tensor.shape)
        masked_tensor = masked_tensor.mask()
        self.assertEquals(tf.reduce_sum(masked_tensor.tensor), 600)
        
        
unittest.main(argv=[''], verbosity=2, exit=False)    


testCompression (__main__.TestDynamic)
Test a compression instance ... ok
testDirectRestore (__main__.TestDynamic)
Tests direct restore. Do I get back out what I put in? ... ok
testMask (__main__.TestDynamic)
Test whether the mask function is working sanely ... C:\Users\chris\AppData\Local\Temp/ipykernel_8428/3295938291.py:101: DeprecationWarning: Please use assertEqual instead.
  self.assertEquals(tf.reduce_sum(masked_tensor.tensor), 600)
FAIL
testResizeRestore (__main__.TestDynamic)
Test resize restore. If I fiddle with the provided dimensions, can I still restore the others? ... ok
testSetup (__main__.TestDynamic)
See if the set up values are sane ... ok
test_isupper (__main__.TestStringMethods) ... ok
test_split (__main__.TestStringMethods) ... ok
test_upper (__main__.TestStringMethods) ... 

tf.Tensor([10.], shape=(1,), dtype=float32)
tf.Tensor([4.], shape=(1,), dtype=float32)
tf.Tensor([5.], shape=(1,), dtype=float32)
tf.Tensor([3.], shape=(1,), dtype=float32)
tf.Tensor([2.], shape=(1,), dtype=float32)
tf.Tensor(1200.0, shape=(), dtype=float32)
[<tf.Tensor: shape=(1,), dtype=float32, numpy=array([4.], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([5.], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([3.], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.], dtype=float32)>, None, None, None, None, None, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([5.], dtype=float32)>]
(1, 1, 1, 1, 10, 4, 5, 3, 2, 1)
tf.Tensor([4.], shape=(1,), dtype=float32)
tf.Tensor([5.], shape=(1,), dtype=float32)
tf.Tensor([3.], shape=(1,), dtype=float32)
tf.Tensor([2.], shape=(1,), dtype=float32)
tf.Tensor([5.], shape=(1,), dtype=float32)


ok

FAIL: testMask (__main__.TestDynamic)
Test whether the mask function is working sanely
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\chris\AppData\Local\Temp/ipykernel_8428/3295938291.py", line 101, in testMask
    self.assertEquals(tf.reduce_sum(masked_tensor.tensor), 600)
AssertionError: <tf.Tensor: shape=(), dtype=float32, numpy=1200.0> != 600

----------------------------------------------------------------------
Ran 8 tests in 0.419s

FAILED (failures=1)


properties check
['d_batch', 'd_rows', 'd_columns', 'd_items', 'd_channels', 'batch', 'rows', 'columns', 'items', 'channels']
(1, 1, 1, 1, 1, 10, 3, 4, 5, 2)
[<tf.Tensor: shape=(1,), dtype=float32, numpy=array([10.], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([3.], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([4.], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([5.], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.], dtype=float32)>, None, None, None, None, None]

compress check
(240, 1, 5)
['d_batch', 'items']
[<tf.Tensor: shape=(1,), dtype=float32, numpy=array([10.], dtype=float32)>, None]
direct restore


In [2]:

class SymTensor():
    """
    A class with a variety of sanity-easing utility functions.
    
    The SymTensor class provides a key-dim corrolation between tensors and their dimensions - 
    effectively allowing the naming and manipulation of dimensions. It's capabilities are modest, but sufficient.
    It allows one to request a reordering of the dimensions in a particular order, a packing of tensor
    dimensions in a reduction, performing updates, and more.
    
    """
    @property
    def tensor(self):
        return self._tensor
    @property
    def names(self):
        return self._names
    @property
    def lengths(self):
        return self._lengths
    def __init__(self, tensor, dim_names):
        """
        The initialization routine. This should store both the tensor, and the dimension
        names. It should be the case that the number of tensor dimensions, and dim_names, is the same
        """
        assert len(tensor.shape) == len(dim_names)
        
        self._tensor = tensor
        self._names = dim_names
        self._lengths = tensor.shape
        
    def compress(self, selection):
        """
        
        Keeps around, in the designated order, everything in selection.
        Everything else gets compressed into one last dimension. Then
        returns the tensor, and a names index.
        """
        
        tensor, names, length = self._reorder(selection)
        reshape = [-1, *tensor.shape[-len(selection):]]
        
        names = names[-len(selection):]
        tensor = tf.reshape(tensor, reshape)
        
        return tensor, names
    def like(self, tensor, definition_names):
        """
        Attempts to take in a tensor, and make a SymTensor "like" it using the information 
        in names. 
        
        Anything provided in names should corrolate to tensor dimensions. These will not be inferred,
        but handled directly. Any dimension which is in tensor, but not the input names, will be inferred to 
        be reshape dimensions. These dimensions will be reshaped to match the inferred dimensions of the current
        instance.
        
        """
        
        #Get names, lengths from current instance. Cut out regions of inference.
        region_width = len(definition_names)
        
        #Get the inferred names and lengths. These are the names and lengths that are NOT
        #defined by the incoming definition names.
        inferred_names, inferred_lengths = [], []
        for name, length in zip(self.names, self.lengths):
            if name not in definition_names:
                inferred_names.append(name)
                inferred_lengths.append(length)
        
        #Get the direct names  and direct lengths. These are defined directly.
        
        direct_names, direct_lengths = definition_names, [*tensor.shape[-region_width:]]
        
        #Create names shape, reshape. Return.
        new_names = inferred_names + direct_names
        new_lengths = inferred_lengths + direct_lengths
        new_tensor = tf.reshape(tensor, new_lengths)
        
        return SymTensor(new_tensor, new_names)
    def reorder(self, selection):
        """
        
        Reorders the tensor to be in a particular order.
        
        """
        
        
        tensor, names, length = self._reorder(selection)
        return SymTensor(tensor, names)
        
    
    def _reorder(self, selection):
        """
        
        A core function. Allows the implicit reordering of the tensor and associated quantities.
        
        Names given are brought to the end of the tensor, the rest are shuffled near the beginning in the current order.
    
        """
        
        #Create permuter. This will rearrange both the tensor, and the list. Do this by looping through names
        #and shuffling them into two catagories - end, which remains ordered as declaration, and start, which is implicit
        
        names = []
        for item in self.names:
            if item not in selection: #Not being shifted to the end
                names.append(item)
                
        names = names + selection
        
        #Convert the permuter to a permute, and use to develop the correct output
        
        lookup = dict(zip(self.names, range(len(self.names))))
        permute = [lookup[name] for name in names]
        tensor = tf.identity(tf.transpose(self.tensor, permute))
        
        #Return results
        
        return tensor, names, tensor.shape
        



test_tensor = tf.zeros([10, 5,  4, 20, 4])
test_names = ["batch", "row", "column", "item", "channel"]
dyn_test = SymTensor(test_tensor, test_names)
print(dyn_test.names)
dyn_test2 = dyn_test.reorder(["row", "column"])
print(dyn_test2.names)
dyn_test3, names = dyn_test2.compress(['row', 'column'])
dyn_test3 = tf.expand_dims(dyn_test3, -1)
print(dyn_test3.shape)
dyn_test4 = dyn_test2.like(dyn_test3, ["row", "column", "local"])
print(dyn_test4.names)


['batch', 'row', 'column', 'item', 'channel']
['batch', 'item', 'channel', 'row', 'column']
(800, 5, 4, 1)
['batch', 'item', 'channel', 'row', 'column', 'local']


In [225]:
test = list(range(5))
index = [0,2]
print(test[index])

TypeError: list indices must be integers or slices, not list

In [36]:
class RaggedOpLib():
    """
    
    There are several operations, principly in the domain of ragged tensors, 
    which I lack the ability to utilize dynamically. 
    
    This is an attempt to get around some of that
    
    
    """
    to_tensor = staticmethod(tf.RaggedTensor.to_tensor)


In [61]:




class Base(keras.layers.Layer):
    """
    
    The base layer. This layer will happily unstack
    a ragged tensor, with ragged dimensions at the front, and forward the result to 
    the method called "forward." It is, naturally, up to the implimentation
    to decide what to do with it from there. 
    
    One thing to note, however, is that when making shape-dependent logic, one should
    use tf.shape, NOT tensor.shape. The latter will only be queried once, while the
    former will build itself into the tensorflow graph.
    
    Both increasing, and decreasing, the depth of the ragged stack are also responsibilities
    the layer assists with. One can define, on initialization, a delta_depth, or d_depth, which
    influences how unstacking and restacking occurs. A positive d_depth will attempt to increase
    the depth by the indicated d_depth. For a negative d_depth, meanwhile, it is vice versa. Both cases will 
    expect changes to the forward method
    
    
    
    
    """
    
    def __init__(self, d_depth=0):
        super().__init__()
        
        self._d_depth = d_depth #positive - make deeper. Negative - make shallower.
 
    def preprocessing(self, tensor, )
    def forward(self, tensor, parameters, *args):
        """
        
        
        """
        raise NotImplimentedError("Forward must be implimented.")
    @tf.function
    def _broken_call(self, tensor, scratchspace, *args):
        """
        Does not work. Cannot compile until I have a better tf.shape function.
        
        """
        
               
        print("compile")
        print(tensor.shape[0])
        
        #Recursion over. Breakout. 
        #Note that the ==() checks for the case of a depth=0 or raggedless tensor.
        #Note that negative d_depth stops recursion early.
        if scratchspace.shape == () or scratchspace.shape[0] == max(-self._d_depth, 0):
            #Recursion depth reached
            if not tf.is_tensor(tensor):
                #Not a tensor, so convert to one.
                tensor = tensor.to_tensor()
            
            return self.forward(tensor, scratchspace, *args)
            
              
        #Initiate and perform recursion. Remember to slice off pieces to account for descent.
        scratch_bypass, scratch_shrank = scratchspace[0], scratchspace[1:]
        tensor_output, scratch_output = [], []     
        for index in tf.range(int(tensor.shape[0])):
            
            item = tensor[index]
            bypass = scratch_bypass[index]
            
            output = self._call(item, bypass)
            result, scratch = self._broken_call(item, bypass)

            
            tensor_output.append(result)
            if scratch is None:
                #Just add the bypass to the stack
                scratch_output.append(bypass)
            else:
                #place the scratch tensor, which should be 2d, on the end of the bypass value,
                #which is 0d.
                scratch_output.append(tf.concat([[bypass], scratch], axis=0))
        
        #Stack results
        print(tensor_output)
        tensor_output = tf.ragged.stack(tensor_output, axis=0)
        scratch_output = tf.concat(scratch_output, axis=0)
        
        #return
        
        return tensor_output, scratch_output
                
    
    
    @tf.function
    def _call(self, tensor, depth, parameters, *args):
        """
        
        
        
        """
        
        
        #Terminate recursion logic.Note the max processing - this stops us early if needed
        #for tensorflow processing
        if depth.shape == () or depth.shape[0] == tf.max(-self._d_depth, 0):
            #Recursion depth reached
            if not tf.is_tensor(tensor):
                #Not a tensor, so convert to one.
                tensor = tensor.to_tensor()
            
            return self.forward(tensor, depth, parameters, *args)
        
        
        #Not ending recursion. Proceed further into chain. Squeeze another dimension off of
        #scratchspace, , 
    
        shape = tensor.shape[:]
        shape = [*shape[1:]] #One dimension will be unpacked

        if self._d_depth > 0:
            shape = [*shape, *[None]*self._d_depth] #One additional dimension for each increase.
        spec = tf.RaggedTensorSpec(shape, dtype=tf.float32)
        
        func = lambda tensor : self._call(tensor, depth[1:], parameters)
        output = tf.map_fn(func, tensor, fn_output_signature=spec)
        
        return output
        #shape = tensor.shape
        #shape = [2, *shape[1:]] #One more dimension, to account for scratch carry dimension. Remove dimension
        #spec = tf.RaggedTensorSpec(shape, dtype=tf.float32) #Remove a 

        #func = lambda tensor : self._call(tensor, scratchspace, *args) #note the depth slice. Reducing count
        #scratch_out, tensor_out = tf.map_fn(func, tensor, fn_output_signature=spec) 
        
        #if isinstance(scratch_out, tf.RaggedTensor):
            #scratch_out = scratch_out.to_tensor()
        
        #Unsqueeze scratchspacek, 
        #while len(scratch_out.shape) > 2:
            #scratch_out = tf.squeeze(scratch_out, axis=0)
                
        
        #return output
    
    def call(self, x, *args):
        """
        
        
        """

        #Unpack depth, tensor. Depth is a 1D tensor, for which the value of shape[0] determines
        #the number of unpack steps to perform. This was needed to allow for compile.
        #
        #Yea, tensorflow is weird.
        
        if tf.is_tensor(x): #Handle startup.
            x = [x, tf.ragged.stack([[]])]
        
        #Unpack. Perform primary processing.
        tensor, depth, parameters = x
        output = self._call(tensor, depth, parameters, *args)
        
        if 
        
        
        

In [62]:
rag = tf.ragged.stack([tf.zeros([10, 2]),tf.zeros([10,1])])
indication = tf.ragged.stack([tf.zeros([2])])
final = [rag, indication]

print(rag)
print(indication)

test = staticmethod(tf.RaggedTensor.to_tensor)
class t():
    pass


class Test(Base):
    def __init__(self):
        super().__init__()
    def forward(self, x, *args):
        return x, None
        
    
test_layer = Test()
test_layer(final)

<tf.RaggedTensor [[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0],
  [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0]],
 [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]]>
<tf.RaggedTensor [[0.0, 0.0]]>
compile
2
compile
None
[tf.RaggedTensor(values=Tensor("while/PartitionedCall_1:0", shape=(None,), dtype=float32), row_splits=Tensor("while/PartitionedCall_1:1", shape=(None,), dtype=int64))]


InaccessibleTensorError: Exception encountered when calling layer "test_17" (type Test).

in user code:

    File "C:\Users\chris\AppData\Local\Temp/ipykernel_7984/1382995202.py", line 82, in _broken_call  *
        tensor_output = tf.ragged.stack(tensor_output, axis=0)

    InaccessibleTensorError: <tf.Tensor 'while/PartitionedCall_1:0' shape=(None,) dtype=float32> is out of scope and cannot be used here. Use return values, explicit Python locals or TensorFlow collections to access it.
    Please see https://www.tensorflow.org/guide/function#all_outputs_of_a_tffunction_must_be_return_values for more information.
    
    <tf.Tensor 'while/PartitionedCall_1:0' shape=(None,) dtype=float32> was defined here:
        File "c:\users\chris\appdata\local\programs\python\python39\lib\runpy.py", line 197, in _run_module_as_main
          return _run_code(code, main_globals, None,
        File "c:\users\chris\appdata\local\programs\python\python39\lib\runpy.py", line 87, in _run_code
          

In [16]:
Base.pack(None, tf.ragged.stack([[3]]), tf.zeros([10, 3, 4, 5, 6]))
Base.pack(None, tf.ragged.stack([[3]]), tf.zeros([10, 3, 4]))
Base.pack(None, tf.ragged.stack([[3]]), tf.zeros([10, 3, 4, 5, 6]))


AttributeError: type object 'Base' has no attribute 'pack'

In [526]:

test = tf.ragged.stack([[4, 6], [4, 5]])
test[:, 0]

ValueError: Cannot index into an inner ragged dimension.

In [502]:
def swapdims(tensor, dim1, dim2):
    """
    A simple function capable of swapping two dimensions.
    
    """
    #Normalize the inputs, such that we are not setting negatives
    length = len(tensor.shape)
    dim1 = dim1 % length
    dim2 = dim2 % length
    
    #Get the shape. Replace dim1 with dim2, dim2 with dim 1. This makes a permutation
    permuter = list(range(length))
    permuter[dim1] = dim2
    permuter[dim2] = dim1
    
    #Permute and return
    
    return tf.transpose(tensor, permuter)

#Test
test = tf.zeros([10,3, 4])
print(swapdims(test, -1, -2).shape)
    

(10, 4, 3)


In [141]:

class Reshape(Base):
    """
    The Reshape Class. The view layer contains code to extend
    the standard tensor reshape mode. It is capable of reshaping the
    last dimension of a tensors shape from one shape to another without
    changing any prior dimensions, so long as the number of tensor entries
    match.

    The reshape Class has two modes. In functional mode, one may provide
    a tensor, input_shape, and output_shape to the method .functional. This
    will then execute the reshape logic. Alternatively, one can instance
    the class with input_shape and output_shape, in which case it will behave
    as a reshaping layer and consistently apply the same reshape.

    For example, if one wanted to reshape tensor t=(4, 3, 4,2), to become (4,3,8),
    one could either initialize a layer with Reshape((4,2), 8), then call it,
    or apply Reshape.functional(t, (4,2), 8)
    """

    ## Define the helper functions which functionally perform the view
    @classmethod
    def functional(cls, tensor, input_shape, output_shape):
        """
        A function to change a tensor with last dimensions
        input shape to last dimensions output shape.

        For instance, given (4, 3, 4,2), reshape (4,2) to (8)
        resulting in (4, 3, 8). Expects tensor to be tensorflow vectors.



        :param tensor: The tensor to reshape
        :param input_shape: The shape of the input dimensions which we wish to handle. Can be an integer, or a list
          of integers.
        :param output_shape: The shape of the output dimensions we wish to end up in
        :return: The reshaped tensor.
        """
        
        #Handle raw integer inputs
        if not isinstance(input_shape, Iterable):
            input_shape = [input_shape]
        if not isinstance(output_shape, Iterable):
            output_shape = [output_shape]

        # Perform reshape. Apply FAFP error handling.
        try:
            slice_length = len(input_shape)  # Find out how many dimensions will be dynamic
            static_shape, replacement_shape = tensor.shape[:-(slice_length)], tensor.shape[-(
                slice_length):]  # Slice apart the input shape.
            reshape = (*static_shape, *output_shape)  # replace the output shape
            return tf.reshape(tensor, reshape)  # reshape and return
        except Exception as err:

            # Perform input verification, and figure out where the failure happened
            msg = "While attempting to reshape, I found an error: %s. \n" % err
            msg = msg + "I was attempting to reshape" + str(input_shape) +  " to " + str(output_shape) + "\n"
            msg = msg + "Holmes adds that: 'Dear Watson, %s'" % cls._autopsy(tensor, input_shape, output_shape)
            raise Exception(msg)

    @classmethod
    def _autopsy(cls, tensor, input_shape, output_shape):
        """
        A helper function to analyze what went wrong after an exception is raised

        :param tensor: The input tensor
        :param input_shape: The input shape
        :param output_shape: The output shape
        :return: A string representing anything I found that is wrong.
        """

        if not tf.is_tensor(tensor):
            return "tensor input was not a torch tensor"

        try:
            input_shape = np.array(input_shape, dtype=np.int32)
            output_shape = np.array(output_shape, dtype=np.int32)
        except:
            return "Either input or output shape were unclean. They did not cast to int array"

        if len(input_shape.shape) > 1:
            return "Input shape had more dimensions than one. Should be an int, or list of ints"
        if len(output_shape.shape) > 1:
            return "Output shape had more dimensions than one. Should be an int, or list of ints"

        if np.prod(input_shape) != np.prod(output_shape):
            return "The number of input tensor units was %s, and out tensor units was %s. These are not compatible" \
                   % (np.prod(input_shape), np.prod(output_shape))

        try:
            tensor + torch.zeros([*input_shape])
        except:
            "Input shape and tensor shape are different"

        return "Could not find any additional information"

    ### Develop the stateful logic for that version of access
    def __init__(self, input_shape, output_shape):
        super().__init__()
        
        self._input_shape = input_shape
        self._output_shape = output_shape

    def __call__(self, tensor):
        output =  self.functional(tensor, self._input_shape, self._output_shape)
        return output 
        

    
#test

test_vec = tf.zeros([10,3,5])
Reshape.functional(test_vec, (3, 5), 15)



<tf.Tensor: shape=(10, 15), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)>

In [ ]:
class Linear(Base):
    """
    
    The Linear class is designed to perform Dense mappings between entries. 
    
    It has several additional extensions to the standard dense case. 
    
    First, it is the case that input tensor size is mutable. Within the specified limits, it
    is possible to provide a tensor that is smaller than specified, in which case only the relevant dimensions
    are engaged.  This works, so long as it is the case that the tensor dimensions does not exceed the 
    capacity of the parameter backend. The output, however, will always be the specified width
    
    Second, the layer is capable of performing complex-shape to complex shape mappings. For instance, one may
    specify
    """
    

In [274]:
class Local(Base):
    """
    
    Create a locally focused view of a tensor, using the given dimension. The local view will end up on the end
    of the tensor. The dimension will almost certaintly also be resized by this process.
    
    Requires an input in (..., item, channel) format to function correctly. 
    Returns an output in (..., item_different, local, channel) format.
    """
    def __init__(self, kernel_width, striding, dilation, target_dim, new_dim):
        
        super().__init__()
        
        self._kernel_length = kernel_width*dilation 
        self._kernel_width = kernel_width 
        self._striding = striding
        self._dilation = dilation
        
        #Names
        
        self._target_dim = target_dim
        self._new_dim = new_dim
        
  
        
    def forward(self, stream, word_spec, *args):
        """
        The logic behind the local tensor. Creates, then performs, a vectorized glimpse, adjusts
        the mean lengths, and returns the localized result.
        
        """
        
        channel_dim, _ = word_spec["channel"]
        item_length = stream.shape[-2]
        
        
        #This is a little bit complex, so deserves some explanation. The extract_glimpse function requires 
        #an input in (batch, row, column, channel) format. It will then extract, based on the input "glimpse offset" parameter
        #views of the input, with one view for each batch. 
        
        #The input is expected to be in a (...batch_stuff, item, channel) format. The plan is to reformat
        #this into a (1, batch_dim_collapsed, item, channel) tensor, then repeat the new batch dim, the one, 
        #the number of times required to get a complete view.
        
        #the stream batches is collapsed into the row column, and this tensor is then repeated for the appropriate number of
        #units on the batch column to provide a unique tensor for each view. After this, it becomes possible to do a vectorized
        #select, then transpose the batch and local dimension, and restore the original shape.
        
        
        #Caclulate the glimpse size and glimpse offset. This is done, taking into account the needs for striding.
        
        unstrided_length = stream.shape[-2] - self._kernel_length
        glimpse_offsets = tf.range(0, unstrided_length, self._striding)
        glimpse_offsets = tf.stack([tf.zeros_like(glimpse_offsets), glimpse_offsets], axis=-1) #Final offsets
        
        final_item_length = glimpse_offsets.shape[0] #Length of final view.
        item_difference = stream.shape[-2] - final_item_length #Difference between initial and final state. Used to update word length
        glimpse_size = tf.constant([final_item_length, self._kernel_length]) #Undilated kernel width. Also, contains logic to perform complete slice.
        
        #Create the needed tensor
        
        shape = [*stream.shape[-2:]] #Get a copy of the streams last two dimensions.
        shape.insert(0, -1) #Insert a collapse direction on the column channel
        shape.insert(0, 1) #Insert a unmodify direction on the batch channel
        tensor = tf.reshape(stream, shape) #Reformat tensor. (1, batch, item, channel).
        tensor = tf.repeat(tensor, final_item_length, axis=0) #Repeat. (final_item, batch, item, channel)
        
        #Create the local glimpses. Perform any dilations

        local_tensor = tf.image.extract_glimpse(tensor, glimpse_size, glimpse_offsets, centered=False, normalized=False) # (final_item, batch, raw_local, channel)
        local_tensor = local_tensor[..., ::self._dilation, :] # Apply dilation. (final_item, batch, local, channel)

        
        #Restore to standard format. 
        local_tensor = swapdims(local_tensor, 0, 1) #swap to (batch, final_item, local, channel) format
        local_tensor = tf.identity(local_tensor) #Ensure the data buffer is reset, with batch on the outside.
        
        shape = [*stream.shape[:-2], *local_tensor.shape[-3:]] #Stitch together original format
        local_tensor = tf.reshape(local_tensor, shape) #expand back to original format.
        
        #Tensor is now in (..., final_item, local, channel) format. 
        
        #Finally, adjust the mean length and return
        
        mean_length = mean_length-self._kernel_length
        return local_tensor, mean_length
        

In [204]:
class Linear

compile
Tensor("inputs:0", shape=(None, None, None), dtype=float32)
None
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 1.03 ms
None


In [162]:
test = tf.range(40)
test = tf.reshape(test, [2, 5, 4])
test = test[tf.newaxis, :, :]
test = tf.cast(test, tf.float32)
test = tf.repeat(test, 2, axis=0)

print(test)
sizes = [2, 2]
offsets = tf.constant([[0, 0], [0, 1]], dtype=tf.float32)

print(tf.image.extract_glimpse(test, sizes, offsets, centered=False, normalized=False))



tf.Tensor(
[[[[ 0.  1.  2.  3.]
   [ 4.  5.  6.  7.]
   [ 8.  9. 10. 11.]
   [12. 13. 14. 15.]
   [16. 17. 18. 19.]]

  [[20. 21. 22. 23.]
   [24. 25. 26. 27.]
   [28. 29. 30. 31.]
   [32. 33. 34. 35.]
   [36. 37. 38. 39.]]]


 [[[ 0.  1.  2.  3.]
   [ 4.  5.  6.  7.]
   [ 8.  9. 10. 11.]
   [12. 13. 14. 15.]
   [16. 17. 18. 19.]]

  [[20. 21. 22. 23.]
   [24. 25. 26. 27.]
   [28. 29. 30. 31.]
   [32. 33. 34. 35.]
   [36. 37. 38. 39.]]]], shape=(2, 2, 5, 4), dtype=float32)
tf.Tensor(
[[[[ 0.  1.  2.  3.]
   [ 4.  5.  6.  7.]]

  [[20. 21. 22. 23.]
   [24. 25. 26. 27.]]]


 [[[ 4.  5.  6.  7.]
   [ 8.  9. 10. 11.]]

  [[24. 25. 26. 27.]
   [28. 29. 30. 31.]]]], shape=(2, 2, 2, 4), dtype=float32)


# Dynamic

Some of the methods of dynamic

Ideas:

* Traveling kernel process
* Frozen pivot process


# Dynamic Item Length

One of the major issues in traditional NLP is the need to statically define the shape of an output. This is an attempt to allow the machine to handle this.


## Process:

* Total vector length does, in fact, remain constant for calculation purposes. We do not need to keep recalculating the 
* graph.
* Items are formatted in [0, item, 0, item, ...] format after embedding process is complete.
* Further layers may inject additional information into internal channels
* Subsequent "resize" layer will take the product of "decide" with process vector:
    [item, item2, item3, item4]*[decide]. This reduces it down to a scalar. 
* The scalar is used to make a decision. Positive scalars are deemed words. Negative ones, ignored.
* The positive cases are extracted, and reembedded as [0, item, 0, item, ...]. 
* This proceeds until end, where loss is calculated. 

## Positive

## Loss

* Loss is calculated as hinge loss. 
* If too many words are active, all active words are met with a rejection penalty. This penalty is equivalent to the inverse of decision* inverse, meaning word which are almost off will shut off faster than words for which we are more certain.
* If too few words are active, the same happens in reverse, with emphasis on words that are off. 
* Backprop or manual gradient adjustment may be necissary.

* Items are always embedded as [0, item, 0, item, 0, item,...] tensors after passing through a special layer.
* This layer in turn 

In [ ]:
class DynamicBase():
    


#  Trainable Dynamic Item Resize by the Technique of Virtual Policy Gradients.

## Dynamic Resize

Dynamic resize consists of allowing the machine learning algorithm to smoothly add or remove "words" or embedding items into it's stream. It is also the case that this should operate according to primary gradient descent to be at all practical. Many, Many challenges are faced here. 

* Smoothness. It should be the case that adding and removing words is numerically "smooth." This means changing the word configuration should NOT result in a large jump 
* Gradients. Gradients should both exist, and be useful at changing the number of words.
* Loss. Loss should be reasonably easy to impliment given the objectives
* Computation. Computation should not be significantly greater than standard methods. Additionally, it should preferably be smaller. 
* Per layer size. It should be the case that the model can decide, per layer, how wide it wants its embeddings to run.

Accomplishing this is by no means an easy feat. Nonetheless, to my suprise, it is possible. The technique to accomplish this is virtual gradients.


Assumption is: Embeddings which are near each other are relavent to each other.



## Frozen Pivot

Frozen pivot gradients is a technique to allow a calculated selector of some sort, in this case the WordLength, to impliment parallel possibilities even in difficult circumstances such as vectors whose sizes are changing. It consists of
a fairly simple update function, the "frozen pivot", which controls how much to prioritize each possibilties. It will only work with ordered possibilities. 

**Forward pass**

On each layer per evaluation, we calculate a WordLength. This WordLength is calculated by running the incoming tensors through several layers, and represents the mean word lenght we expect to output. After this, we use it to calculate new numbers +-epsilon, etc. This is then known as the compression tensor, and represents how many words the next layer should have. 

After this we turn the Representation tensor into the Proxy tensor. First, we develop a linear equation centered at the mean and governed by y =(m_d-m) * x + m_d, where m is the graph mean and m_d is the mean, but detatched. This is then first fed through a softmax function, creating the Proxy tensor. Also, do note it is absolutely essencial that the gradient be halted on m_d. Finally, we use the compression tensor to reshape the input, then run them through layers, and finally combine them with probabilities given by the Proxy tensor. The output, and the WordLength are then returned. 

Note that at this point, we are actually done. Any attempt by the backprop algorithm to modify Proxy will inevitably ONLY have the ability to modify the mean. So long as a longer, or shorter, tensor allows better results the model should see it and expand or contract as needed. Additionally, the mean itself will also be pushed to be smaller, or larger, by the regularization parameters.

**Loss And Regularization**

Primary loss consists of the normal calculation process.

Meanwhile, several additional regularization and loss terms are required to make the algorithm work.

* A endpoint loss for yelling at the model for having too few or too many words. 
* A regularization loss, L2?, on the mean for each layer. Encourages compression.
* A strong penalty for any mean whose words are dropping below zero.
* Dropout on the mean construction layer. Introduces jitter which ensures the model width does not lock up.


The regularization loss is a simple matter. Gather the means, and scale. The endpoint loss will also, it turns out, be fairly easy. Simply go ahead ahead and form a loss with respect to the WordLength, which is returned for that exact purpose, and should dominate. All other behavior can be left to the model, which should quickly rescale it's output to the appropriate size, then slowly fill in model sizes as the process proceeds.



**Expand-Mean Resize**

The LCM-Mean Resize operates as follows. 

t



The LCM-Mean Resize is an algorithm which attempts to preserve local context between embedding entries. It operates as follows. Let there be a length L that the embeddings currently exist as, and a length K which is the target size.

## Resize Transfomer

The action of resize needs equal consideration to the action of training the resize. While Frozen Pivot may make training a rescale possible, consideration still needs to be given to how to make it actually happen. This is where tranformers and the resize logic comes into play.

Transformers will be utilized to resize the layers, but that still leaves a problem: Generating the queue. While it will be the case that the Key and Value can be generated from the incoming stream directly, the Queue needs to be the length of the 
output to work correctly. 



**Queue**

The queue is generated by performing an interleave expansion, a view, and then either a max or a mean. 

Let K be the initial embeddings size, and L be the target embeddings size. Perform a repeat interleave along embeddings of length L. The embeddings now cleanly divide into L. Go and perform a view on the embeddings, looking at (L, K). It is now the case that one dimension is the right shape. 

At this point, any summarizing statistic can be performed on the last embedding dimension to get something of length L. Mean is predicted to be particularly helpful when operating locally, but max might be useful as well.

**Transformer**

With a Queue tensor made, Transfomer can be executed. Take the Queue, take the Content, expand heads and transform. Resize is then complete. 


In [ ]:
class Resize 

In [38]:
class Penalty():
    """
    
    The penalty class is an extremely important portion of the
    dynamic layer. It is responsible for collecting a penalty 
    forcing a corrolation between selectors. Several types of penalty exist. 
    Currently, these are Corrolation and Jacobian.

    Corrolation computes the logits version of a corrolation, then uses hinge
    loss to compute a penalty. It is required for the corrolation to be above 
    a certain amount for no penalty to exist
    
    Jacobian penalty, meanwhile, computes the Jacobian between 
    the selections. The Jacobian entries then become an indication of corrolation.
    Any entry which is below a certain threshold is then penalized
    """
    @staticmethod
    def covarience(a, b):
        
        #flatten most of the dimensions into a single batch. Calculate N
        a = a.view([-1, a.shape[-1]]) 
        b = b.view([-1, b.shape[-1]])
        n = a.shape[-2]
        
        #Calculate the covarience between the channel items.
        
        cov = (a-a.mean(-1).unsqueeze(-1))(b - b.mean(-1).unsqueeze(-1))/n
        
        #Return the covarience.
        return cov 

    def __init__(self, variety="Covarience", threshold =0.2):
        """
        The init function. 
        
        Variety may be "Covarience" or "Jacobian." Threshold controls
        how strong the hinge loss is.
        
        """
    def __call__(self, select_old, select_new, penalty)

class Dynamic()
    """
    
    Dynamic is a metalayer containing the layers and functions
    needed to do Dynamic Resize in functional, procedural, and 
    object oriented manners. 
    
    The three primary classes in dynamic are Select, 
    Encode, and Scatter. These, respectively, are in charge of selecting words to move forward with,
    encoding the unselected residuals back into the selection tensor, and then scattering the output into a 
    InterspacedWord. The classes are part of the metaclass, and may be initialized individually - Additionally,
    it is the case that the subclasses have a functional method, which can be used to execute the logic directly.
    
    The driving observation which makes this layer possible is that it is possible to built a follow up operator such 
    that so long as the transition between having x and x+1 tensors occurs when the magnitude of the tensor is zero, there
    is little to no influence on the output, driving it's relation into a linear zone. To do this, we smooth the 
    output with a no-bias local transformer, though both biased and unbiased are included for the sake of experimentation.    
    """
    ## Classes. To reduce clutter to some degree, I store a number of classes inside the larger class.
    

    ### Helper: These are called by primary logic functions. ###
    
    @staticmethod
    def lcm_mean_reshape(tensor, dim, length):
        """

        Perform a weightless resize, attempting to preserve some degree of
        local information

        """
        #place the dim in the correct location

        tensor = tensor.swapdims(-1, dim)

        #Calculate the lcm of the length and the dimension length. Use this to calculate kernel size. 

        dim_length = tensor.shape[-1]
        lcm = math.lcm(dim_length, length)
        expansion_multiplier = lcm//dim_length
        kernel_size = lcm//length


        #Expand kernel to size. Perform local view of appropriate dimensions. Average views. return.
        kernel = tensor.repeat_interleave(expansion_multiplier, dim=-1)
        width = kernel_size - 1
        stride = width

        kernel = LocalView.functional(kernel, 0, width, kernel_size, 1, dim=-1, pad=True)

        print(kernel.shape)
        kernel = kernel.mean(dim=-1).squeeze()

        output = kernel.swapdims(-1, dim)

        return output
    
    ### Primary Logic: These are called in sequence from forward ###
    
    def select(cls, tensor):
        """
        The select metod is a functional method designed to produce a selector given a cutoff
        function. 
        
        A selector is a tensor with a single scalar value for each entry in channel dimension. This value
        should be positive, if the word will be retained, or negative, if not.
        """
        return self._cutoff(selector)
    
    
  
    @classmethod
    def segment(cls, tensor, selector):
        """
        The segment method is responsible for taking in a tensor and a selector, then utilizing it 
        to create a set of rejected and accepted tensors. 
        
        It does this with masks and sort. Tensors have a decision made about them, then masked into two groups and
        are sorted by which decisive catagory they fell into. After this, they are sliced to the longest tensor,
        and the two groups are returned. 
        
        
        """
        
        # Develop the sort tensor, which represents retained values with a one. 
        #Also, multiply the tensor by the selector, ensuring a smooth transition region.
        
        sort_in = torch.where(selector > 0, 1, 0)
        tensor = selector*tensor



        #Find the number of active, off words. Find their indices using sort. Figure out the length of the output
        #vector which will be needed.

        active = (selector > 0).sum().type(torch.int32)
        off = (selector < 0).sum().type(torch.int32)
        _, primary_indices = torch.sort(sort_in, dim=-2, descending=True, stable=True) #Should not change order. Active first.
        _, rejection_indices = torch.sort(sort_in, dim=-2, descending=False, stable=True)


        #Create the reformatted generation tensor. Give the program the ability to influence how much a word is on. Do this
        #by multiplying the tensor by the selector, and hence encouraging vectors to be zero when near off. Then split into
        #two tensors; the primary and residual tensor.

        primary = torch.where(selector > 0, tensor, 0)
        rejections = torch.where(selector < 0, tensor, 0)

        #Sort the primary tensor. Cut it out. Sort the residual tensor. Cut them out. Then use a lcm-mean to embed
        #the residual information back into the primary tensor, allowing gradient descent to continue acting upon it.

        #It should be noted that this particular trick is VERY important. Since residuals are "relatively local" and since
        #the relu trick means that getting closer to the decision threshold shuts off each tensor, an output item can turn "off"
        #more either by reducing it's actual input, or increasing how strongly off it's residual sources are. Likewise, it may turn 
        #more strongly on both by increasing it's magnitude, AND by bringing a residual instance more online. 

        #Furthermore, since at weak scales it is the case that under near-zero conditions linear effects dominate,
        #per the taylor expansion, the transition will be quite smooth, sans any trouble due to positional embeddings.

        primary = primary[primary_indices]
        primary = primary[..., :active, :]

        rejections = rejections[residual_indices]
        rejections = rejections[..., :off, :]
        
        return primary, rejections
        

    def scatter(self, scatter_tensor):
        """
        This is the  scatter method.
        
        The scatter method takes in a tensor to scatter, and the amount of interspace, 
        then proceeds to make a wordchange tensor with the scatters throughout it. It does
        not, however, retain the residual information.

        """

        interspace = self._interspace
        #Put tensor in item, channel dimension format

        scatter_tensor = scatter_tensor.swapdims(-1, channel_dim)
        scatter_tensor = scatter_tensor.swapdims(-2, item_dim)

        #Calculate, then create, a tensor big enough to be returned. The tensor should have the 
        #properties that it has a distance of interspace before the first word, then alternate words
        #and interspace distance

        scatter_length = scatter_tensor.shape[-2]
        output_length = interspace + scatter_length + scatter_length*interspace
        build_shape = scatter_length.shape[:]
        build_shape[-2] = output_length()

        output = torch.zeros(built_shape)

        #Perform a strided scatter into output. Start after the first interspace, and then place the entire scatter
        #tensor into the output, striding by the interspace plus one distance

        output[(interspace+1)::(interspace+1)] = scatter_tensor

        #Rearrange output, then return

        output = output.swapdims(-1, channel_dim)
        output = output.swapdims(-2, item_dim)
        return output
    @classmethod
    def _residual_encode(cls, tensor, residual, alpha):
        """
        
        The residual encode method is responsible for giving gradient descent something to push against when 
        encoding.
        
        It takes in the output tensor, and the rejections, then combines them such that 
        a weak output occurs even on words deemed to be "off," in the manner of a leaky relu
        
        """
        required_length = tensor.shape[-2]
        reformatted_residuals = self.lcm_mean_reshape(residual, dim=-2, required_length)    
        return tensor+alpha*reformatted_residuals

    def __init__(self, internal, channel_width, cutoff, oenalty, smooth,
                 interspace=1, channel_dim=-1, item_dim = -2, alpha=0.01, beta=0.01):
        
        #Start up torch
        super().__init__()
        
        #
        self._channel_dim = channel_dim
        self._item_dim = item_dim
        
        #Store layers.

        self.internal = internal
        self.smooth = smooth
        self._cutoff = cutoff
        self.penalty = penalty
        
        #Store constants
        self._interspace = interspace
        self._alpha = alpha
        self._beta = beta
        
        #Create layernorm
        
        self._LayerNorm = nn.LayerNorm((channel_width))
        
    
    def process(self, stream, selection):
        """
        process takes in a tensor, and a selection. It then returns a tensor and a selection.
        """
        
        ## Perform primary processing. 
        
        tensor, residual = self.segment(stream, selection) #Segment
        tensor = self.scatter(tensor) #Scatter
        tensor = self.smooth(tensor) #Smooth.
        tensor = self.internal(tensor) #Sublayer logic
        tensor = self.residual_encode(tensor, residual, -self._alpha) #Off residual passthrough
        tensor = self.residual_encode(tensor, stream, self._beta) #On residual passthrough
        tensor = self.LayerNorm(tensor)
        
        #Create selection
        select = self.select(tensor)
        
        #Return outputs 
        return tensor, select
        
    def forward(self, stream, select, penalty=None):
        
        tensor, selector = self.process(stream, select)
        penalty += self.penalty(stream, selector)
    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 253)

In [11]:
tensor = torch.zeros((10, 4, 3))
tensor.masked_select(torch.tensor([True, False, True])).shape

torch.Size([80])

In [35]:

import torch
noise_a = 10*torch.rand([20, 10])
noise_b = torch.rand([3, 10])

def corrolating(a, b):
    

print(corrolating(noise_a, noise_b))



torch.Size([23, 10])
torch.Size([23, 23])
torch.Size([3, 20])
tensor([[-0.3247, -0.2765,  0.1318, -0.3361,  0.3723,  0.1311, -0.1954,  0.5786,
          0.4519, -0.2662, -0.0466,  0.5212, -0.1485, -0.3027,  0.6917, -0.0187,
         -0.5208,  0.1179,  0.0363, -0.0585],
        [-0.0030, -0.0390,  0.2266,  0.0653,  0.1567, -0.1725, -0.2575,  0.0119,
          0.4352, -0.0791, -0.0950, -0.1559,  0.2093, -0.7495,  0.1910,  0.6346,
         -0.2099,  0.0363,  0.0903, -0.0086],
        [ 0.0301,  0.1271, -0.3464,  0.3308, -0.2540,  0.0055, -0.2179, -0.3874,
         -0.2600, -0.0021,  0.0254, -0.3917, -0.1571, -0.1900, -0.3285, -0.0197,
          0.3219, -0.0585, -0.0086,  0.0824]])


# 